In [1]:
import csv
import numpy as np
import pandas as pd
import time
from urllib2 import urlopen # python2
import urllib2

In [7]:
sider_id = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/3_3_SIDER_STITCHid_DrugBankid.tsv")
sider_se = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/3_4_SIDER_db_side_effect.tsv")
offside_id = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/4_3_OFFSIDE_STITCHid_DrugBankid.tsv")
offside_se = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/4_4_OFFSIDE_db_side_effect.tsv")

In [5]:
df_asso = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/3_PREDICT_id_association.tsv")

In [10]:
print(len(df_asso))
print(len(df_asso.drop_duplicates(subset='DrugBank_ID')))
print(len(df_asso.drop_duplicates(subset='OMIM_ID')))
df_asso.head(2)

1920
588
313


,DrugBank_ID,OMIM_ID
0,DB00659,103780
1,DB00284,125853


In [26]:
sider_merged = pd.merge(left=sider_id, right=sider_se)

In [29]:
sider_db_se = sider_merged[['DrugBank_ID','UMLS_concept_id_for_MedDRA','side_effect_name']]
print len(sider_db_se)
print len(sider_db_se.drop_duplicates())

73951
73951


In [61]:
sider_db_se.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/3_5_SIDER_DrugBank_sideffect.tsv", sep='\t', index=False)
sider_db_se.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/3_5_SIDER_DrugBank_sideffect.tsv", sep='\t', index=False)

In [23]:
off_merged = pd.merge(left=offside_id, right=offside_se, left_on = 'STITCH_compound_id_stereo', right_on='stitch_id')

In [24]:
off_db_se = off_merged[['DrugBank_ID', 'umls_id', 'event']]
print len(off_db_se)
print len(off_db_se.drop_duplicates())

195460
195460


In [62]:
off_db_se.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/4_5_OFFSIDE_DrugBank_sideffect.tsv", sep='\t', index=False)
off_db_se.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/4_5_OFFSIDE_DrugBank_sideffect.tsv", sep='\t', index=False)

In [30]:
sider_db_se = sider_db_se.rename(columns={'UMLS_concept_id_for_MedDRA':'umls_id', 'side_effect_name':'event'})

In [52]:
concated = pd.concat([sider_db_se, off_db_se])
print len(concated)

269411


In [53]:
drugbank_se = concated.drop_duplicates(['DrugBank_ID', 'umls_id'])

In [50]:
print len(drugbank_se)
print len(drugbank_se.drop_duplicates('DrugBank_ID'))
print len(drugbank_se.drop_duplicates('umls_id'))
print len(drugbank_se.drop_duplicates('event'))

259679
1034
10009
12209


In [64]:
drugbank_se.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/5_DrugBank_SideEffect_total.tsv", sep='\t', index=False)
drugbank_se.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/5_DrugBank_SideEffect_total.tsv", sep='\t', index=False)

In [58]:
print len(df_asso)
df_asso_se = df_asso[df_asso['DrugBank_ID'].isin(drugbank_se['DrugBank_ID'])]
print len(df_asso_se)
print len(df_asso_se.drop_duplicates('DrugBank_ID'))
print len(df_asso_se.drop_duplicates('OMIM_ID'))

1920
1554
460
282


In [59]:
gold_se = drugbank_se[drugbank_se['DrugBank_ID'].isin(df_asso['DrugBank_ID'])]
print len(gold_se)
print len(gold_se.drop_duplicates())
print len(gold_se.drop_duplicates('DrugBank_ID'))
print len(gold_se.drop_duplicates('umls_id'))

158296
158296
460
9092


In [66]:
gold_se.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/PREDICT_data/1_gold_drug_se.tsv", sep='\t', index=False)
gold_se.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/PREDICT_data/1_gold_drug_se.tsv", sep='\t', index=False)

In [35]:
len(df_sider.drop_duplicates(subset=['STITCH_compound_id_flat','STITCH_compound_id_stereo']))

787

In [15]:
print(len(df_offside))
df_offside.head()

558


,STITCH_compound_id_stereo,DrugBank_ID
0,CID000000119,DB02530
1,CID000000338,DB00936
2,CID000000444,DB01156
3,CID000000564,DB00513
4,CID000000596,DB05494


In [19]:
print len(df_asso[df_asso['DrugBank_ID'].isin(df_sider['DrugBank_ID'])].drop_duplicates(subset='DrugBank_ID'))
df_asso[df_asso['DrugBank_ID'].isin(df_sider['DrugBank_ID'])].drop_duplicates(subset='DrugBank_ID').head()

374


,DrugBank_ID,OMIM_ID
5,DB01193,608622
7,DB00316,145590
12,DB00819,147480
24,DB00210,604416
25,DB00640,194200


In [20]:
print len(df_asso[df_asso['DrugBank_ID'].isin(df_offside['DrugBank_ID'])].drop_duplicates(subset='DrugBank_ID'))
df_asso[df_asso['DrugBank_ID'].isin(df_offside['DrugBank_ID'])].drop_duplicates(subset='DrugBank_ID').head()

278


,DrugBank_ID,OMIM_ID
0,DB00659,103780
5,DB01193,608622
7,DB00316,145590
12,DB00819,147480
23,DB00787,268320


In [21]:
a = pd.concat([df_sider['DrugBank_ID'], df_offside['DrugBank_ID']])

In [23]:
b = a.drop_duplicates()

In [29]:
print len(df_sider['DrugBank_ID'].drop_duplicates())
print len(df_offside['DrugBank_ID'].drop_duplicates())
print len(a)
print len(b)

801
558
1359
1034


In [28]:
print len(df_asso['DrugBank_ID'].drop_duplicates())
print len(df_asso[df_asso['DrugBank_ID'].isin(b)].drop_duplicates(subset='DrugBank_ID'))
print len(df_asso)
print len(df_asso[df_asso['DrugBank_ID'].isin(b)])
df_asso[df_asso['DrugBank_ID'].isin(b)].drop_duplicates(subset='DrugBank_ID').head()

588
460
1920
1554


,DrugBank_ID,OMIM_ID
0,DB00659,103780
5,DB01193,608622
7,DB00316,145590
12,DB00819,147480
23,DB00787,268320


In [32]:
c = df_asso[df_asso['DrugBank_ID'].isin(b)]
print(len(df_asso.drop_duplicates(subset='OMIM_ID')))
print(len(c.drop_duplicates(subset='OMIM_ID')))

313
282
